In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import datetime

In [2]:
stock_name = "ADANIENT"
from_date = "01/11/2021"
to_date = "01/11/2022"

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--disable-gpu')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)

driver.get('https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?flag=0&type=ETF')
print(driver.title)

driver.find_element(By.XPATH, "//input[@value='rad_no1']").click()
search = driver.find_element(By.ID, "ContentPlaceHolder1_smartSearch")
search.send_keys(stock_name)
search.send_keys(Keys.RETURN)

start_date = datetime.datetime.strptime(from_date, "%d/%m/%Y")
end_date = datetime.datetime.strptime(to_date, "%d/%m/%Y")

chunk_size = 26

current_date = start_date
columns = ["Date","Open","High","Low","Close","WAP","No. of Shares","No. of Trades","Total Turnover"]
df = pd.DataFrame(columns=columns)
while current_date < end_date:
    date_from = current_date.strftime("%d/%m/%Y")
    date_to = (current_date + datetime.timedelta(days=chunk_size - 1)).strftime("%d/%m/%Y")

    date_input = driver.find_element(By.ID, "ContentPlaceHolder1_txtFromDate")
    driver.execute_script("arguments[0].value = '{}';".format(date_from), date_input)

    date_input = driver.find_element(By.ID, "ContentPlaceHolder1_txtToDate")
    driver.execute_script("arguments[0].value = '{}';".format(date_to), date_input)

    date_input.send_keys(Keys.RETURN)
    submit_button = driver.find_element(By.ID, "ContentPlaceHolder1_btnSubmit")
    submit_button.click()

    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_spnStkData")))

    table_html = table.get_attribute('outerHTML')

    soup = BeautifulSoup(table_html, 'html.parser')

    data = []
    for row in soup.find_all('tr', class_='TTRow'):
        date = row.find('td', class_='TTRow_left10').text.strip()
        values = [td.text.replace(',', '') for td in row.find_all('td', class_='tdcolumn text-right')][0:8]
        data.append([date] + values)

    df_chunk = pd.DataFrame(data, columns=columns)

    df = pd.concat([df, df_chunk], ignore_index=True)

    current_date += datetime.timedelta(days=chunk_size)

driver.quit()


Stock Prices


In [4]:
df

,Date,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover
0,1/11/21,1444.50,1456.65,1411.05,1446.00,1435.15,110033,3587,157913467
1,2/11/21,1451.30,1488.30,1451.30,1472.45,1474.40,96753,4447,142652327
2,3/11/21,1472.45,1500.00,1465.00,1486.20,1480.75,52850,3132,78257394
3,4/11/21,1499.90,1500.00,1483.20,1497.20,1496.75,40905,1509,61224635
4,8/11/21,1494.70,1575.85,1485.30,1567.35,1534.77,176244,8378,270493778
...,...,...,...,...,...,...,...,...,...
261,21/11/22,4017.00,4017.00,3920.80,4001.10,3977.34,63917,6360,254219676
262,22/11/22,4014.00,4056.00,3987.30,4035.60,4032.26,41648,3543,167935652
263,23/11/22,4048.10,4060.00,3873.05,3903.65,3949.53,131366,10350,518833384
264,24/11/22,3912.20,3935.00,3822.00,3922.00,3882.71,109450,9763,424963030


In [5]:
df = df.iloc[0:0]
df

,Date,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover
